In [1]:
import torch
import sys

sys.path.append('/home/afahim2/tmp/clip-project/clipverse')


from clips.clip_assembler import ClipAssembler
from src.evaluator import Evaluator
from dataset_processors.mscoco_processor import MSCOCOProcessor

from src.prepare_config import prepare_config
from clips.hf_clip import HFClipOutput, HFClip

/home/afahim2/miniconda3/envs/babylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import importlib
importlib.reload(sys.modules['dataset_processors'].mscoco_processor)
importlib.reload(sys.modules['src'].evaluator)
importlib.reload(sys.modules['src'].prepare_config)

<module 'src.prepare_config' from '/home/afahim2/tmp/clip-project/clipverse/src/prepare_config.py'>

In [2]:
cuda_device = 'cuda:0'
ROOT = '/home/afahim2/tmp/clip-project/clipverse'

In [3]:
mscoco_paths = {
        # 128D

        # Seed 42
        'mscoco_default_128d_s42': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lit_42_finetune_I1C2E1E2_128_val_as_val_512_mscoco_VIT_pretrained_EVAL.pt',
        'mscoco_cua_128d_s42': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_42_finetune_I1C2E1E2_128_val_as_val_512_mscoco_VIT_pretrained_FINAL3.pt',
        'mscoco_cuaxu_128d_s42': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_xuniform_42_finetune_I1C2E1E2_128_val_as_val_512_mscoco_VIT_pretrained_EVAL.pt',

        # Seed 24

        'mscoco_default_128d_s24': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lit_24_finetune_I1C2E1E2_128_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cua_128d_s24': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_24_finetune_I1C2E1E2_128_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cuaxu_128d_s24': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_xuniform_24_finetune_I1C2E1E2_128_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',

        # Seed = 44
        'mscoco_default_128d_s44':  '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lit_44_finetune_I1C2E1E2_128_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cua_128d_s44':  '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_44_finetune_I1C2E1E2_128_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cuaxu_128d_s44': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_xuniform_44_finetune_I1C2E1E2_128_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',

        # 64D

        'mscoco_default_64d_s42': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lit_42_finetune_I1C2E1E2_64_val_as_val_512_mscoco_VIT_pretrained_EVAL.pt',
        'mscoco_cua_64d_s42': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_42_finetune_I1C2E1E2_64_val_as_val_512_mscoco_VIT_pretrained_FINAL3.pt',
        'mscoco_cuaxu_64d_s42': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_xuniform_42_finetune_I1C2E1E2_64_val_as_val_512_mscoco_VIT_pretrained_EVAL.pt',

        'mscoco_default_64d_s24': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lit_24_finetune_I1C2E1E2_64_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cua_64d_s24': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_24_finetune_I1C2E1E2_64_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cuaxu_64d_s24': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_xuniform_24_finetune_I1C2E1E2_64_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',

        'mscoco_default_64d_s44': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lit_44_finetune_I1C2E1E2_64_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cua_64d_s44': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_44_finetune_I1C2E1E2_64_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cuaxu_64d_s44': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_xuniform_44_finetune_I1C2E1E2_64_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',

        # 32D

        'mscoco_default_32d_s42': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lit_42_finetune_I1C2E1E2_32_val_as_val_512_mscoco_VIT_pretrained_EVAL.pt',
        'mscoco_cua_32d_s42': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_42_finetune_I1C2E1E2_32_val_as_val_512_mscoco_VIT_pretrained_FINAL3.pt',
        'mscoco_cuaxu_32d_s42': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_xuniform_42_finetune_I1C2E1E2_32_val_as_val_512_mscoco_VIT_pretrained_EVAL.pt',

        'mscoco_default_32d_s24':  '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lit_24_finetune_I1C2E1E2_32_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cua_32d_s24': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_24_finetune_I1C2E1E2_32_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cuaxu_32d_s24': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_xuniform_24_finetune_I1C2E1E2_32_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',

        'mscoco_default_32d_s44': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lit_44_finetune_I1C2E1E2_32_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cua_32d_s44': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_44_finetune_I1C2E1E2_32_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',
        'mscoco_cuaxu_32d_s44': '/remote/cirrus-home/afahim2/tmp/clip-project/clipverse/checkpoints/T0.01_Lituniform_align_xuniform_44_finetune_I1C2E1E2_32_val_as_val_512_mscoco_VIT_pretrained_FINAL10.pt',

    }

### Load Dataset

In [4]:
val_batch_cache_file = f'{ROOT}/datasets/mscoco/val_batch_cache_mscoco_full_5k.pt'


### Evaluate CLIP models

In [6]:

def get_gap_stuff(evaluator: Evaluator, clip_model: HFClip):
    # ranks = evaluator.get_rank(linalg=False)
   

    return {
        'mean_cosine_similarity': evaluator.get_mean_cosine_similarity(clip_model.get_temperature()),
        'linear_seperability_accuracy': evaluator.get_linear_seperability(),
        'centroid_euclidean_distance': evaluator.get_centroid_euclidean_distance(),

        'val_image_classification_acc': evaluator.get_val_image_classification_acc(return_all=True),

        'get_val_image_retrieval_acc': evaluator.get_val_image_retrieval_acc(return_all=True),

        # 'image_variances': ranks['image_explained_variance_ratios'],
        # 'text_variances': ranks['text_explained_variance_ratios'],

        # 'uniformity_loss': evaluator.get_mscoco_uniformity(),
        # 'alignment_loss': evaluator.get_mscoco_alignment(),
        
    }

In [7]:

device = torch.device(cuda_device if torch.cuda.is_available() else 'cpu')

gap_metrics = {}
# have same keys in gap_metrics as in mscoco_paths

with torch.no_grad():


    for model_name, model_path in mscoco_paths.items():
        print(' --- ')
        print(f'Processing {model_name}')
        print()
        dimensionality = int(model_name.split("_")[2].split("d")[0])

        prepare_config(clip_projection_dim=dimensionality, cuda_device=cuda_device, seed=2, wandb_enabled=False)
        loaded_clip_model = ClipAssembler().clip_model.to(device)
        checkpoint = torch.load(model_path, map_location=device)
        model_state_dict = checkpoint['model_state_dict']
        loaded_clip_model.load_state_dict(model_state_dict)

        loaded_clip_model.eval()
        loaded_clip_model.half()



        evaluator = Evaluator(MSCOCOProcessor(root=ROOT), val_batch_cache_file, load_train_dataset=False)

        evaluator.set_val_outputs(loaded_clip_model, output_loss=False)
        evaluator.set_outputs_to_use('val')

        gap_stuff = get_gap_stuff(evaluator, loaded_clip_model)

        gap_metrics[model_name] = gap_stuff

 --- 
Processing mscoco_default_128d_s42



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.



--- ENCODER 1 = IMAGE --- 


 --- Initializing CLIP Image Encoder from pretrained model ---


 --- Changing projection layer size of CLIP Image Encoder: VIT  to 128--- 


 --- Unfreezing backbone weights of CLIP Image Encoder: VIT  --- 


--- ENCODER 2 = TEXT --- 


 --- Initializing Text Encoder with CLIP tokenizer from pretrained model ---


 --- Changing projection layer size of Text Encoder with CLIP tokenizer to 128  --- 


 --- Unfreezing backbone weights of Text Encoder with CLIP tokenizer --- 

CLIP device  cuda
check point path for CLIP model  checkpoints/T0.01_Lit_2_finetune_I1C2E1E2_128_val_as_val_5000_mscoco_VIT_pretrained_POST_PAPER.pt
CLIP model not loaded from checkpoint

--- HF CLIP MODEL ---

selected clip model  FINETUNED_TEMP
temperature (T):  0.01


In [ ]:
val_batch_cache_file = f'{ROOT}/datasets/mscoco/val_batch_cache_mscoco_full_5k.pt'

mscoco_val_imgs: torch.Tensor = None
mscoco_val_captions: list[str] = None

(mscoco_val_imgs, mscoco_val_captions) = torch.load(val_batch_cache_file)